---
## **📊 Flow**   
> step01 : Data   
> - 데이터 준비 및 분석   
> - 데이터 전처리
   #   
> setp02 : 비교 모델    
> - ResNet50, EfficientNet,VGG16
> - 모델 학습
> - 성능평가 : 정확도 재현율 f1
   #
> setp03 : 성능 개선   
> ResNet50, EfficientNet,VGG16 모델들 **앙상블**   
> 성능평가

---
> ### step01 : Data   
> - 데이터 준비 및 분석   
> - 데이터 전처리

In [9]:
from dataprocessing import *

# 데이터 경로
data_path = '/home/dibaeck/sketch/study_Data4Quality/task02_CovidClassifier/COVID19_1K'

---
>> 데이터 사이즈 다름 --> TASK : 데이터 리사이즈  

In [ ]:
check_image_sizes(data_path)

---
>> 클래스 불균형 : COVID19 데이터가 적음.  --> TASK : Data Augmentation   

In [ ]:
count_and_visualize_images(data_path)

클래스 불균형 해결 전략 : WeightedRandomSampler + FocalLoss

| 전략               | 설명                                                                 |
|--------------------|----------------------------------------------------------------------|
| **WeightedRandomSampler**          | 수 클래스를 더 자주 뽑히게 하는 샘플링 방식                |
| **Focal Loss**     | 소수 클래스의 어려운 샘플에 더 집중하는 loss function(의료 이미지에서 성능 개선에 효과적)                               |

---

In [6]:
######################################## setting 
from dataprocessing import *
import torch
from torch.utils.data import DataLoader, WeightedRandomSampler

# 데이터 경로
data_path = '/home/dibaeck/sketch/study_Data4Quality/task02_CovidClassifier/COVID19_1K'

# 데이터 정규화 및 리사이즈, 텐서화
transform = get_transform()

train_dataset, val_dataset, test_dataset = get_datasets(data_path, transform)

# 데이터 로더 batch size 32
train_loader = DataLoader(train_dataset, shuffle=True)
val_loader = DataLoader(val_dataset, shuffle=False)
test_loader = DataLoader(test_dataset, shuffle=False)

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
import numpy as np

# 모델 훈련 및 평가 함수
def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='binary' if len(set(y_test)) == 2 else 'micro')
    recall = recall_score(y_test, y_pred, average='binary' if len(set(y_test)) == 2 else 'micro')
    f1 = f1_score(y_test, y_pred, average='binary' if len(set(y_test)) == 2 else 'micro')
    confusion = confusion_matrix(y_test, y_pred)
    
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"Confusion Matrix:\n{confusion}")

# 이진 분류를 위한 데이터 준비 (PNEUMONIA vs NORMAL 등)
def prepare_data_for_binary_classification(loader, class_1_idx, class_2_idx):
    data = []
    labels = []
    for inputs, targets in loader:
        # 특정 클래스에 해당하는 것만 필터링
        binary_labels = torch.where(targets == class_1_idx, 1, 0)  # class_1_idx를 1로, 나머지는 0으로
        data.append(inputs.view(inputs.size(0), -1).numpy())  # 이미지를 1D 벡터로 변환
        labels.append(binary_labels.numpy())
    return np.concatenate(data), np.concatenate(labels)

# 다중 클래스 분류를 위한 데이터 준비
def prepare_data_for_multi_classification(loader):
    data = []
    labels = []
    for inputs, targets in loader:
        data.append(inputs.view(inputs.size(0), -1).numpy())  # 이미지를 1D 벡터로 변환
        labels.append(targets.numpy())
    return np.concatenate(data), np.concatenate(labels)

# 모델 훈련 및 성능 비교 함수
def train_and_evaluate_models(train_loader, test_loader):
    # 1) PNEUMONIA vs NORMAL (이진 분류)
    X_train_pneumonia, y_train_pneumonia = prepare_data_for_binary_classification(train_loader, class_1_idx=0, class_2_idx=1)
    X_test_pneumonia, y_test_pneumonia = prepare_data_for_binary_classification(test_loader, class_1_idx=0, class_2_idx=1)
    
    # 모델: 로지스틱 회귀
    model_pneumonia = RandomForestClassifier(n_estimators=100)
    model_pneumonia.fit(X_train_pneumonia, y_train_pneumonia)
    
    print("PNEUMONIA vs NORMAL Model Performance:")
    evaluate_model(model_pneumonia, X_test_pneumonia, y_test_pneumonia)

    # 2) COVID19 vs NORMAL (이진 분류)
    X_train_covid, y_train_covid = prepare_data_for_binary_classification(train_loader, class_1_idx=1, class_2_idx=2)
    X_test_covid, y_test_covid = prepare_data_for_binary_classification(test_loader, class_1_idx=1, class_2_idx=2)
    
    model_covid = RandomForestClassifier(n_estimators=100)
    model_covid.fit(X_train_covid, y_train_covid)
    
    print("\nCOVID19 vs NORMAL Model Performance:")
    evaluate_model(model_covid, X_test_covid, y_test_covid)

    # 3) Multi-class: COVID19 vs PNEUMONIA vs NORMAL (다중 클래스 분류)
    X_train_multi, y_train_multi = prepare_data_for_multi_classification(train_loader)
    X_test_multi, y_test_multi = prepare_data_for_multi_classification(test_loader)
    
    model_multi = RandomForestClassifier(n_estimators=100)
    model_multi.fit(X_train_multi, y_train_multi)
    
    print("\nMulti-class Model Performance (COVID19 vs PNEUMONIA vs NORMAL):")
    evaluate_model(model_multi, X_test_multi, y_test_multi)

In [13]:
# 모델 훈련 및 평가 실행
train_and_evaluate_models(train_loader, test_loader)

PNEUMONIA vs NORMAL Model Performance:
Accuracy: 0.9655
Precision: 0.9286
Recall: 0.6842
F1-Score: 0.7879
Confusion Matrix:
[[183   1]
 [  6  13]]

COVID19 vs NORMAL Model Performance:
Accuracy: 0.9360
Precision: 0.9744
Recall: 0.7600
F1-Score: 0.8539
Confusion Matrix:
[[152   1]
 [ 12  38]]

Multi-class Model Performance (COVID19 vs PNEUMONIA vs NORMAL):
Accuracy: 0.9064
Precision: 0.9064
Recall: 0.9064
F1-Score: 0.9064
Confusion Matrix:
[[ 12   1   6]
 [  1  39  10]
 [  1   0 133]]


### 모델 성능 비교

| **모델**                          | **정확도 (Accuracy)** | **정밀도 (Precision)** | **재현율 (Recall)** | **F1-Score** | **Confusion Matrix**                                |
|-----------------------------------|----------------------|------------------------|---------------------|--------------|-----------------------------------------------------|
| **PNEUMONIA vs NORMAL**           | 0.9655               | 0.9286                 | 0.6842              | 0.7879       | [[183   1] <br> [  6  13]]                         |
| **COVID19 vs NORMAL**             | 0.9360               | 0.9744                 | 0.7600              | 0.8539       | [[152   1] <br> [ 12  38]]                         |
| **Multi-class (COVID19 vs PNEUMONIA vs NORMAL)** | 0.9064 | 0.9064 | 0.9064 | 0.9064 | [[ 12   1   6] <br> [  1  39  10] <br> [  1   0 133]] |

---

### 결론

- **PNEUMONIA vs NORMAL 모델**: 높은 정확도와 정밀도, 하지만 재현율이 상대적으로 낮습니다.
- **COVID19 vs NORMAL 모델**: 높은 정확도, 정밀도, 재현율을 보이며, 비교적 균형 잡힌 성능을 보여줍니다.
- **Multi-class 모델**: 세 가지 클래스에 대해 고른 성능을 보여주며, F1-Score와 정확도가 90% 이상으로 우수합니다.

**최종 평가**:
- **COVID19 vs NORMAL 모델**은 정밀도와 재현율이 균형을 이루어 가장 우수한 성능을 보입니다.
- **Multi-class 모델**은 세 가지 클래스를 잘 분리하며 좋은 성능을 보였습니다.

---

1]Multi-class분류에서 사용된 loss 가 모든 클래스에 적절했는지 분석하고,Focal loss,class-balancedloss등을 적용한 실험을 수행해보세요.

1) Loss 함수 분석 및 Focal Loss, Class-Balanced Loss 실험   
Multi-class 분류에서 기본적으로 사용되는 loss 함수는 Cross-Entropy Loss입니다. 이를 Focal Loss나 Class-Balanced Loss로 변경하여 성능을 비교할 수 있습니다.
   
Focal Loss:   
Focal Loss는 클래스 불균형 문제를 해결하기 위한 loss 함수입니다. Cross-Entropy Loss는 클래스 간 불균형이 심할 때 성능이 떨어질 수 있는데, Focal Loss는 어려운 샘플에 가중치를 주어 성능을 개선할 수 있습니다.

In [19]:
import torch
import torch.nn as nn

class FocalLoss(nn.Module):
    def __init__(self, gamma=2, alpha=0.25, num_classes=3):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.num_classes = num_classes
        self.ce_loss = nn.CrossEntropyLoss(reduction='none')

    def forward(self, inputs, targets):
        ce_loss = self.ce_loss(inputs, targets)
        pt = torch.exp(-ce_loss)  # For each class
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()
    
class ClassBalancedLoss(nn.Module):
    def __init__(self, class_weights):
        super(ClassBalancedLoss, self).__init__()
        self.class_weights = class_weights
        self.ce_loss = nn.CrossEntropyLoss(weight=self.class_weights)

    def forward(self, inputs, targets):
        return self.ce_loss(inputs, targets)
    
from sklearn.metrics import precision_score, recall_score

def adjust_threshold(predictions, targets, threshold=0.5):
    pred_classes = (predictions[:, 1] > threshold).float()  # COVID-19 클래스의 확률을 기준으로 예측
    precision = precision_score(targets, pred_classes, average='binary', pos_label=1)
    recall = recall_score(targets, pred_classes, average='binary', pos_label=1)
    return precision, recall

import cv2
import numpy as np
import torch
from torchvision import models

class GradCAM:
    def __init__(self, model):
        self.model = model
        self.model.eval()
        self.gradients = None
        self.activations = None

    def save_gradient(self, grad):
        self.gradients = grad

    def forward(self, x):
        activations = self.model.features(x)
        activations.register_hook(self.save_gradient)
        return activations

    def generate_gradcam(self, input_image, target_class):
        activations = self.forward(input_image)
        self.model.zero_grad()
        activations[target_class].backward(retain_graph=True)
        gradients = self.gradients
        pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
        weighted_activations = activations * pooled_gradients.view(1, -1, 1, 1)
        gradcam = torch.mean(weighted_activations, dim=1).squeeze()
        gradcam = np.maximum(gradcam.cpu().detach().numpy(), 0)
        gradcam = cv2.resize(gradcam, (input_image.size(2), input_image.size(3)))
        return gradcam

import shap
import numpy as np

# SHAP을 사용하여 모델 해석
def explain_with_shap(model, dataloader):
    explainer = shap.KernelExplainer(model.predict, data=dataloader)
    shap_values = explainer.shap_values(dataloader)
    shap.summary_plot(shap_values, dataloader)

from sklearn.metrics import confusion_matrix

def analyze_confusion_matrix(predictions, targets):
    cm = confusion_matrix(targets, predictions)
    print("Confusion Matrix:\n", cm)

# 예시: 오분류된 샘플에 대해 가중치를 더 주기
def reweight_loss_function(model, predictions, targets):
    # 잘못 분류된 샘플에 대해 가중치 증가
    incorrect_samples = predictions != targets
    weights = torch.where(incorrect_samples, 2.0, 1.0)
    loss = F.cross_entropy(predictions, targets, weight=weights)
    return loss


/home/dibaeck/sketch/anaconda3/envs/dibk311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
# 모델 학습 시 사용될 loss 함수 설정
criterion = FocalLoss(gamma=2, alpha=0.25, num_classes=3)  # 예시로 FocalLoss 사용

# 학습 과정에서 해당 loss를 사용하여 모델 훈련


In [23]:
thresholds = [0.3, 0.5, 0.7]  # 여러 threshold 값 시도
for threshold in thresholds:
    precision, recall = adjust_threshold(predictions, targets, threshold)
    print(f"Threshold: {threshold} -> Precision: {precision}, Recall: {recall}")


NameError: name 'predictions' is not defined

In [22]:
gradcam = GradCAM(model)
cam_output = gradcam.generate_gradcam(input_image, target_class)
# 결과를 시각화
plt.imshow(cam_output, cmap='jet')
plt.show()


NameError: name 'model' is not defined